### Search Engine With Tools And Agents

In [8]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)

   ------------- -------------------------- 1/3 [beautifulsoup4]
   ---------------------------------------- 3/3 [wikipedia]

Note: you may need to restart the kernel to use updated packages.


In [10]:
## Arxiv--Research(website)
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [11]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [12]:
pip install arxiv

^C
Note: you may need to restart the kernel to use updated packages.


In [13]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [14]:
tools=[wiki,arxiv]

In [15]:
pip install langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables

True

In [17]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [18]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

In [21]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [22]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [23]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever

C:\Users\sande\AppData\Local\Temp\ipykernel_7708\3897781561.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F7989AC260>, search_kwargs={})

In [24]:
#to convert retriever into a tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

In [25]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\sande\\Desktop\\Genai\\env\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [28]:
pip install langchain_groq

  Using cached langchain_groq-0.3.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached groq-0.24.0-py3-none-any.whl.metadata (15 kB)
Using cached langchain_groq-0.3.2-py3-none-any.whl (15 kB)
Using cached groq-0.24.0-py3-none-any.whl (127 kB)

   ---------------------------------------- 0/2 [groq]
   ---------------------------------------- 2/2 [langchain_groq]

Note: you may need to restart the kernel to use updated packages.


In [29]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")


llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [30]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
## Agents-----chain
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [32]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [33]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was found
Invoking: `arxiv` with `{'query': 'Langsmith'}`


Published: 2020-10-09
Title: Langsmith: An Interactive Academic Text Revision System
Authors: Takumi Ito, Tatsuki Kuribayashi, Masatoshi Hidaka, Jun Suzuki, Kentaro Inui
Summary: Despite the current diversity and inclusion initiatives in the academicBased on the result, I can provide a direct response. Langsmith is an interactive academic text revision system that helps authors revise their texts to make them more accessible and inclusive. It was published in 2020 and has authors from various institutions.

> Finished chain.


{'input': 'Tell me about Langsmith',
 'output': 'Based on the result, I can provide a direct response. Langsmith is an interactive academic text revision system that helps authors revise their texts to make them more accessible and inclusive. It was published in 2020 and has authors from various institutions.'}

In [34]:
agent_executor.invoke({"input":"Tell me about REVA UNIVERSITY"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'REVA University'}`


Page: REVA University
Summary: REVA University is a private university in Kattigenahalli, Yelahanka, Bangalore. It was established under the Government of Karnataka Act, 2012. It is managed by the Rukmini Educational Charitable Trust. The university 
Invoking: `wikipedia` with `{'query': 'REVA University#Academics'}`


Page: REVA University
Summary: REVA University is a private university in Kattigenahalli, Yelahanka, Bangalore. It was established under the Government of Karnataka Act, 2012. It is managed by the Rukmini Educational Charitable Trust. The university It seems that the tool call for "call_y5jp" did not yield any additional information. Since we already have the basic information about REVA University, I can provide a direct response to the user.

REVA University offers undergraduate and postgraduate programs in various disciplines such as engineering, management, commerce, 

{'input': 'Tell me about REVA UNIVERSITY',
 'output': 'It seems that the tool call for "call_y5jp" did not yield any additional information. Since we already have the basic information about REVA University, I can provide a direct response to the user.\n\nREVA University offers undergraduate and postgraduate programs in various disciplines such as engineering, management, commerce, humanities, and sciences. The university is also known for its research initiatives and has collaborations with industries and other institutions.'}

In [36]:
agent_executor.invoke({"input":"Autism disorder"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Autism spectrum disorder'}`


Page: Autism
Summary: Autism spectrum disorder (ASD), or simply autism, is a neurodevelopmental disorder characterized by repetitive, restricted, and inflexible patterns of behavior, interests, and activities, as well as difficulties in social intera
Invoking: `wikipedia` with `{'query': 'Autism symptoms'}`


Page: Classic autism
Summary: Classic autism, also known as childhood autism, autistic disorder, or Kanner's syndrome, is a formerly diagnosed neurodevelopmental disorder first described by Leo Kanner in 1943. It is characterized by atypical and impa
Invoking: `wikipedia` with `{'query': 'Autism signs and symptoms'}`


Page: Classic autism
Summary: Classic autism, also known as childhood autism, autistic disorder, or Kanner's syndrome, is a formerly diagnosed neurodevelopmental disorder first described by Leo Kanner in 1943. It is characterized by atypical and impa
Invokin

{'input': 'Autism disorder',
 'output': 'Here is the response:\n\nAutism diagnosis presents a major challenge due to the vast heterogeneity of the condition. However, researchers have been exploring various methods to improve the accuracy of diagnosis, including the analysis of human gestures and gait. This study presents a novel approach for detecting autism by analyzing human gestures and gait patterns. The proposed method uses machine learning algorithms to classify individuals with autism from those without the condition. The results show that the proposed method can accurately identify individuals with autism with a high degree of accuracy. This study has significant implications for the development of more effective diagnosis tools for autism.'}